In [1]:
import pandas as pd
import numpy as np
import h5py
# import geopandas as gpd
# import numpy as np
# from shapely.geometry import Point, MultiPoint
# from shapely.ops import nearest_points
# from shapely.geometry import Point
# import matplotlib.pyplot as plt

In [138]:
# Load household data that has been joined to the nearest parcel with residential land uses
hh = pd.read_csv(r'C:\Users\Brice\Desktop\gis\hh_14_nearest_parcel.csv')

In [139]:
# Drop the building info, it's not necessarily correct
hh.drop(['building_i', u'building_t',
       u'improvemen', u'job_capaci', u'land_area', u'non_reside',
       u'not_demoli', u'residentia', u'sqft_per_u', u'stories',
       u'template_i', u'year_built'], axis=1, inplace=True)

In [140]:
# Let's just assume that the join correctly placed the households, without doing any
# other spatial joins

parcel = pd.read_csv(r'R:\Urbansim\parcel_14.csv')
bldg = pd.read_csv(r'R:\Urbansim\bldg_14.csv')

# Get building totals per parcel
bldg_total = bldg.groupby('parcel_id').sum()[[u'improvement_value',u'job_capacity','non_residential_sqft',
                                             'residential_units','stories']].reset_index()
bldg_avg = bldg.groupby('parcel_id').sum()[['sqft_per_unit']].reset_index()

# Merge bldg_totals and averages
bldg_agg = pd.merge(bldg_total,bldg_avg,on='parcel_id')

# Join this to the household file
df = pd.merge(hh,bldg_agg,on='parcel_id',how='left')

# join with parcel data to get land value
df = pd.merge(df,parcel,on='parcel_id', how='left')

In [141]:
# bldg_avg
# [i for i in df.columns]

In [142]:
# Calculate percent of parcel building space for residential use
df['residential_sqft'] = df['sqft_per_unit']*df['residential_units']
df['total_sqft'] = df['residential_sqft']+df['non_residential_sqft']
df['total_value'] = df[u'improvement_value']+df['land_value']
df['percent_residential_sqft'] = df['residential_sqft']/df['total_sqft']
df['total_residential_value'] = df['percent_residential_sqft']*df['total_value']
df['unit_residential_value'] = df['total_residential_value']/df['residential_units']

In [260]:
df.to_csv(r'R:\\urbansim\hh_14_with_parcel_info.csv',index=False)

In [143]:
df_survey = df.copy()

# Compare Synthetic HH to Housing Costs

In [252]:
# Load synthetic hh
myh5 = h5py.File(r'R:\SoundCast\Inputs\lodes\vision\2014\landuse\hh_and_persons.h5')

In [253]:
hh_synth_df = pd.DataFrame()
for col in myh5['Household'].keys():
    hh_synth_df[col] = myh5['Household'][col][:]

IOError: Can't read data (File read failed: time = wed nov 07 15:47:18 2018
, filename = 'r:\soundcast\inputs\lodes\vision\2014\landuse\hh_and_persons.h5', file descriptor = 3, errno = 22, error message = 'invalid argument', buf = 0000000017145950, total read size = 49, bytes this sub-read = 49, bytes actually read = 18446744073709551615, offset = 14753783)

In [ ]:
# Join this to the household file
df = pd.merge(hh_synth_df,bldg_agg,left_on='hhparcel',right_on='parcel_id',how='left')

# join with parcel data to get land value
df = pd.merge(df,parcel,on='parcel_id', how='left')

In [ ]:
# Calculate percent of parcel building space for residential use
df['residential_sqft'] = df['sqft_per_unit']*df['residential_units']
df['total_sqft'] = df['residential_sqft']+df['non_residential_sqft']
df['total_value'] = df[u'improvement_value']+df['land_value']
df['percent_residential_sqft'] = df['residential_sqft']/df['total_sqft']
df['total_residential_value'] = df['percent_residential_sqft']*df['total_value']
df['unit_residential_value'] = df['total_residential_value']/df['residential_units']

In [22]:
df.to_csv(r'R:\\urbansim\hh_synth_14_with_parcel_info.csv',index=False)

In [ ]:
df_synth = df.copy()

## Join survey to synthetic households for comparison

In [182]:
# Calculate avg and median housing costs for survey and synth
df_survey_median = df_survey.groupby('hh_income_detailed_imp').median()[['unit_residential_value']].reset_index()
df_survey_median.rename(columns={'unit_residential_value':'measure'})
df_survey_median['measure'] = 'unit_residential_value_median'
df_survey_median['source'] = 'survey'

df_survey_mean = df_survey.groupby('hh_income_detailed_imp').mean()[['unit_residential_value']].reset_index()
df_survey_mean.rename(columns={'unit_residential_value':'measure'})
df_survey_mean['measure'] = 'unit_residential_value_mean'
df_survey_mean['source'] = 'survey'

df = df_survey_median.append(df_survey_mean)

# bin household income for synthetic population


In [183]:
income_bins = [-1,10000,25000,35000,50000,75000,100000,150000,200000,250000,999999999]
income_labels = [1,2,3,4,5,6,7,8,9,10]

In [184]:
df_synth['hh_income_detailed_imp'] = pd.cut(df_synth['hhincome'], bins=income_bins, labels=income_labels)

In [185]:
df_synth_median = df_synth.groupby('hh_income_detailed_imp').median()[['unit_residential_value']].reset_index()
df_synth_median.rename(columns={'unit_residential_value':'measure'})
df_synth_median['measure'] = 'unit_residential_value_median'
df_synth_median['source'] = 'synthetic_pop'

df = df.append(df_synth_median)

df_synth_mean = df_synth.groupby('hh_income_detailed_imp').mean()[['unit_residential_value']].reset_index()
df_synth_mean.rename(columns={'unit_residential_value':'measure'})
df_synth_mean['measure'] = 'unit_residential_value_mean'
df_synth_mean['source'] = 'synthetic_pop'

df = df.append(df_synth_mean)

In [65]:
df.to_csv(r'R:\\urbansim\compare_survey_synth_14.csv',index=False)

In [186]:
survey_synth_compare = df.copy()

# Evaluate future year income-costs

In [187]:
# Get housing costs from urbansim
bldgf = pd.read_csv(r'R:\urbansim\bldg_2050_jf_run_45.run_2018_07_06_11_05.csv.csv')
parcel = pd.read_csv(r'R:\urbansim\parcel_2050_jf_run_45.run_2018_07_06_11_05.csv.csv')

In [188]:
# Get hh info from the run
hh_df = pd.read_csv(r'R:\urbansim\hh_2050_jf_run_45.run_2018_07_06_11_05.csv')
hh_df.drop([u'residential_units', u'sqft_per_unit', u'stories',
            'non_residential_sqft','improvement_value'],axis=1,inplace=True)

In [189]:
# Get building totals per parcel
bldg_total = bldg.groupby('parcel_id').sum()[[u'improvement_value',u'job_capacity','non_residential_sqft',
                                             'residential_units','stories']].reset_index()
bldg_avg = bldg.groupby('parcel_id').sum()[['sqft_per_unit']].reset_index()

# Merge bldg_totals and averages
bldg_agg = pd.merge(bldg_total,bldg_avg,on='parcel_id')

In [190]:
# Join this to the household file
df = pd.merge(hh_df,bldg_agg,on='parcel_id',how='left')

# Already have parcel info
# # join with parcel data to get land value
# df = pd.merge(df,parcel,on='parcel_id', how='left')

In [191]:
# Calculate percent of parcel building space for residential use
df['residential_sqft'] = df['sqft_per_unit']*df['residential_units']
df['total_sqft'] = df['residential_sqft']+df['non_residential_sqft']
df['total_value'] = df[u'improvement_value']+df['land_value']
df['percent_residential_sqft'] = df['residential_sqft']/df['total_sqft']
df['total_residential_value'] = df['percent_residential_sqft']*df['total_value']
df['unit_residential_value'] = df['total_residential_value']/df['residential_units']

In [192]:
# # For forecast households, we know the exact building ID, so join based on that

# # Merge parcel data to building data -> to be connected to household data
# bldg_parcel = pd.merge(bldg_df, parcel_df, on='parcel_id', how='left')

# # Join data to household information
# hh_df = pd.merge(hh, bldg_parcel, on='building_id', how='left')

# # Calculate housing cost
# df = hh_df.copy()

In [193]:
# df


In [194]:
# df.income

In [195]:
# Record income
income_bins = [-1,10000,25000,35000,50000,75000,100000,150000,200000,250000,999999999]
income_labels = [1,2,3,4,5,6,7,8,9,10]
df['hh_income_detailed_imp'] = pd.cut(df['income'], bins=income_bins, labels=income_labels)

In [130]:
df.to_csv('R:\urbansim\hh_50_jf_run_45.csv', index=False)

In [196]:
df_2050 = df.copy()

In [199]:


# Group to get totals by income groups
df_2050['hh_income_detailed_imp'] = pd.cut(df_2050['income'], bins=income_bins, labels=income_labels)

df_2050_median = df_2050.groupby('hh_income_detailed_imp').median()[['unit_residential_value']].reset_index()
df_2050_median.rename(columns={'unit_residential_value':'measure'})
df_2050_median['measure'] = 'unit_residential_value_median'
df_2050_median['source'] = '2050_just_friends'

survey_synth_compare = survey_synth_compare.append(df_2050_median)

df_2050_mean = df_2050.groupby('hh_income_detailed_imp').mean()[['unit_residential_value']].reset_index()
df_2050_mean.rename(columns={'unit_residential_value':'measure'})
df_2050_mean['measure'] = 'unit_residential_value_mean'
df_2050_mean['source'] = '2050_just_friends'

survey_synth_compare = survey_synth_compare.append(df_2050_mean)

In [200]:
survey_synth_compare.to_csv('R:\urbansim\compare_14survey_synth_2050.csv', index=False)

In [202]:
# survey_synth_compare

# Compare Movers Only

In [203]:
# Compare against households from 2045
# Are they locating in a way we'd expect from the 2014 synth pop?

In [206]:
# Get housing costs from urbansim
bldgf = pd.read_csv(r'R:\urbansim\bldg_2040_jf_run_40.run_2018_07_06_11_05.csv.csv')
parcel = pd.read_csv(r'R:\urbansim\parcel_2040_jf_run_45.run_2018_07_06_11_05.csv.csv')

# Get hh info from the run
hh_df = pd.read_csv(r'R:\urbansim\hh_2040_jf_run_40.run_2018_07_06_11_05.csv')
hh_df.drop([u'residential_units', u'sqft_per_unit', u'stories',
            'non_residential_sqft','improvement_value'],axis=1,inplace=True)

In [226]:
# hh_df.columns

In [228]:
df_50_40 = pd.merge(df_2050[['household_id','building_id']], hh_df[['building_id','household_id']], 
                    on='building_id', how='left', suffixes=['_50','_40'])

In [249]:
# Get a list of households that have different building than in 2040
moved_since_2040 = df_50_40[df_50_40['household_id_40'].isnull()]['household_id_50'].values

In [254]:
# Get a list of households that have different building than in 2014

# Get housing costs from urbansim
bldgf = pd.read_csv(r'R:\urbansim\bldg_2014_jf_run_40.run_2018_07_06_11_05.csv.csv')
parcel = pd.read_csv(r'R:\urbansim\parcel_2040_jf_run_45.run_2018_07_06_11_05.csv.csv')

# Get hh info from the run
hh_df = pd.read_csv(r'R:\urbansim\hh_2014_jf_run_40.run_2018_07_06_11_05.csv')
hh_df.drop([u'residential_units', u'sqft_per_unit', u'stories',
            'non_residential_sqft','improvement_value'],axis=1,inplace=True)



In [256]:
df_50_14 = pd.merge(df_2050[['household_id','building_id']], hh_df[['building_id','household_id']], 
                    on='building_id', how='left', suffixes=['_50','_14'])
# Get a list of households that have different building than in 2040
moved_since_2014 = df_50_14[df_50_14['household_id_14'].isnull()]['household_id_50'].values

In [257]:
len(moved_since_2014)

959016

In [258]:
len(moved_since_2040)

207896

In [262]:
# Look up the 2050 location for these households and see what the spread looks like
df_2050_movers_since_14 = df_2050[df_2050['household_id'].isin(moved_since_2014)]
df_2050_movers_since_40 = df_2050[df_2050['household_id'].isin(moved_since_2040)]

In [277]:
# Join with existing dataset to get totals
compare_all = survey_synth_compare.copy()

In [283]:
compare_all.groupby('source').count()

,hh_income_detailed_imp,unit_residential_value,measure
source,,,
2050_just_friends,20,20,20
2050_movers_since_2014,20,20,20
2050_movers_since_2040,20,20,20
survey,20,20,20
synthetic_pop,20,20,20


In [278]:
# Group to get totals by income groups
df_2050_movers_since_14_median = df_2050_movers_since_14.groupby('hh_income_detailed_imp').median()[['unit_residential_value']].reset_index()
df_2050_movers_since_14_median.rename(columns={'unit_residential_value':'measure'})
df_2050_movers_since_14_median['measure'] = 'unit_residential_value_median'
df_2050_movers_since_14_median['source'] = '2050_movers_since_2014'

compare_all = compare_all.append(df_2050_movers_since_14_median)

df_2050_movers_since_14_mean = df_2050_movers_since_14.groupby('hh_income_detailed_imp').mean()[['unit_residential_value']].reset_index()
df_2050_movers_since_14_mean.rename(columns={'unit_residential_value':'measure'})
df_2050_movers_since_14_mean['measure'] = 'unit_residential_value_mean'
df_2050_movers_since_14_mean['source'] = '2050_movers_since_2014'

compare_all = compare_all.append(df_2050_movers_since_14_mean)

In [279]:
# Now add movers since 2040
# Group to get totals by income groups
df_2050_movers_since_40_median = df_2050_movers_since_40.groupby('hh_income_detailed_imp').median()[['unit_residential_value']].reset_index()
df_2050_movers_since_40_median.rename(columns={'unit_residential_value':'measure'})
df_2050_movers_since_40_median['measure'] = 'unit_residential_value_median'
df_2050_movers_since_40_median['source'] = '2050_movers_since_2040'

compare_all = compare_all.append(df_2050_movers_since_40_median)

df_2050_movers_since_40_mean = df_2050_movers_since_40.groupby('hh_income_detailed_imp').mean()[['unit_residential_value']].reset_index()
df_2050_movers_since_40_mean.rename(columns={'unit_residential_value':'measure'})
df_2050_movers_since_40_mean['measure'] = 'unit_residential_value_mean'
df_2050_movers_since_40_mean['source'] = '2050_movers_since_2040'

compare_all = compare_all.append(df_2050_movers_since_40_mean)

In [280]:
compare_all.to_csv(r'R:\urbansim\compare_all_plus_movers.csv')

# Compare Building Values 2014 to 2050

In [312]:
# Get housing costs from urbansim
bldg_parcel_14 = pd.read_csv(r'R:\urbansim\bldg_parcel_2014_jf_run_45.run_2018_07_06_11_05.csv')
bldg_parcel_50 = pd.read_csv(r'R:\urbansim\bldg_parcel_2050_jf_run_45.run_2018_07_06_11_05.csv')

df = bldg_parcel_14.copy()

# Calculate value of the single building
df['residential_sqft'] = df['sqft_per_unit']*df['residential_units']
df['total_sqft'] = df['residential_sqft']+df['non_residential_sqft']
df['total_value'] = df[u'improvement_value']+df['land_value']
df['percent_residential_sqft'] = df['residential_sqft']/df['total_sqft']
df['total_residential_value'] = df['percent_residential_sqft']*df['total_value']
df['unit_residential_value'] = df['total_residential_value']/df['residential_units']

bldg_parcel_14 = df.copy()

df = bldg_parcel_50.copy()

# Calculate value of the single building
df['residential_sqft'] = df['sqft_per_unit']*df['residential_units']
df['total_sqft'] = df['residential_sqft']+df['non_residential_sqft']
df['total_value'] = df[u'improvement_value']+df['land_value']
df['percent_residential_sqft'] = df['residential_sqft']/df['total_sqft']
df['total_residential_value'] = df['percent_residential_sqft']*df['total_value']
df['unit_residential_value'] = df['total_residential_value']/df['residential_units']

bldg_parcel_50 = df.copy()

In [314]:
bldg_parcel_14['unit_residential_value'].mean()

395764.8341251715

In [313]:
bldg_parcel_50['unit_residential_value'].mean()

485133.5385870429

In [318]:
# ID New buildings
new_bldgs = bldg_parcel_50[-bldg_parcel_50['building_id'].isin(bldg_parcel_14['building_id'].values)]
new_bldgs['year'] = 'new bldgs only'

In [316]:
bldg_parcel_14['year'] = '2014'
bldg_parcel_50['year'] = '2050'

In [317]:
df = bldg_parcel_14.append(bldg_parcel_50)
df = df.append(new_bldgs)

In [323]:
new_bldgs['unit_residential_value'].mean()

1123820.965406643

In [322]:
df.to_csv(r'R:\Urbansim\bldg_data.csv',index=False)

In [392]:
bldg_parcel_50['unit_residential_value'].mean()

485133.5385870429

In [333]:
# Convert state plane to lat long geogpandas
import pandas as pd
import geopandas
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [332]:
# Calculate
df = bldg_parcel_50.copy()
df['Coordinates'] = list(zip(df.x_coord_sp, df.y_coord_sp))

In [334]:
df['Coordinates'] = df['Coordinates'].apply(Point)

In [335]:
gdf = geopandas.GeoDataFrame(df, geometry='Coordinates')

In [360]:
gdf.crs = {'init' :'epsg:2285'}

In [361]:
gdf_latlon = gdf.to_crs({'init': 'epsg:4326'})

In [362]:
gdf_latlon['x'] = gdf_latlon.Coordinates.x
gdf_latlon['y'] = gdf_latlon.Coordinates.y

In [368]:
# df.census_block_group_id

In [364]:
gdf_latlon = gdf_latlon[['x','y','building_id','parcel_id']]

In [365]:
gdf_latlon.to_csv(r'R:\urbansim\bldg_parcel_50_coordiantes.csv',index=False)

In [369]:
df.columns

Index([u'building_id', u'building_type_id', u'improvement_value',
       u'job_capacity', u'land_area', u'non_residential_sqft', u'not_demolish',
       u'parcel_id', u'residential_units', u'sqft_per_unit', u'stories',
       u'template_id', u'year_built', u'_init_error_ln_land_value',
       u'census_block_group_id', u'census_block_id', u'city_id', u'county_id',
       u'elem_id', u'faz_group_id', u'faz_id', u'grid_id', u'growth_center_id',
       u'hschool_id', u'is_in_transit_zone',
       u'is_inside_urban_growth_boundary', u'is_waterfront',
       u'land_use_type_id', u'land_value', u'large_area_id', u'mix_split_id',
       u'mschool_id', u'parcel_sqft', u'parking_price_daily',
       u'parking_price_hourly', u'parking_space_daily',
       u'parking_space_hourly', u'plan_type_id', u'regional_geography_id',
       u'school_district_id', u'tractcity_id', u'use_code', u'x_coord_sp',
       u'x_coord_utm', u'y_coord_sp', u'y_coord_utm', u'zip_id', u'zone_id',
       u'residential_sqft

In [376]:
# Spatial join with the census block group shapefile or tract?
tract_shp = geopandas.read_file(r'W:\geodata\census\Tract\tract2010.shp', encoding="utf-8")

In [377]:
tract_shp.crs = {'init' :'epsg:2285'}
# spatial join

In [379]:
df_with_tract = geopandas.sjoin(gdf, tract_shp, how='inner', op='intersects')

In [381]:
# join the tract name to the file we export
df_out = pd.merge(gdf_latlon, df_with_tract[['GEOID10','parcel_id']], on='parcel_id')

In [384]:
df_out.to_csv(r'R:\urbansim\bldg_parcel_50_coordiantes.csv',index=False)

In [385]:
bldg_parcel_50.columns

Index([u'building_id', u'building_type_id', u'improvement_value',
       u'job_capacity', u'land_area', u'non_residential_sqft', u'not_demolish',
       u'parcel_id', u'residential_units', u'sqft_per_unit', u'stories',
       u'template_id', u'year_built', u'_init_error_ln_land_value',
       u'census_block_group_id', u'census_block_id', u'city_id', u'county_id',
       u'elem_id', u'faz_group_id', u'faz_id', u'grid_id', u'growth_center_id',
       u'hschool_id', u'is_in_transit_zone',
       u'is_inside_urban_growth_boundary', u'is_waterfront',
       u'land_use_type_id', u'land_value', u'large_area_id', u'mix_split_id',
       u'mschool_id', u'parcel_sqft', u'parking_price_daily',
       u'parking_price_hourly', u'parking_space_daily',
       u'parking_space_hourly', u'plan_type_id', u'regional_geography_id',
       u'school_district_id', u'tractcity_id', u'use_code', u'x_coord_sp',
       u'x_coord_utm', u'y_coord_sp', u'y_coord_utm', u'zip_id', u'zone_id',
       u'residential_sqft

In [390]:
bldg_df[bldg_df['building_id'] == 1210408].T

,1114452
building_id,1210408
building_type_id,19
improvement_value,219376
job_capacity,-1
land_area,12704
non_residential_sqft,0
not_demolish,0
parcel_id,785649
residential_units,1
sqft_per_unit,2079
